# Compute trajectories for Greenland

In [1]:
import numpy as np
import pylab as plt
import xarray as xr
from joblib import Parallel, delayed
import geopandas as gp
from pathlib import Path
from pism_ragis.trajectories import compute_trajectory, compute_perturbation, get_perturbed_velocities
from pism_ragis.processing import tqdm_joblib
import pandas as pd
from pyDOE import lhs
from scipy.stats.distributions import uniform
from tqdm.auto import tqdm

In [2]:
data_url = Path("~/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc")
ogr_url = Path("/Users/andy/Google Drive/My Drive/data/GreenlandFluxGatesAschwanden/greenland-flux-gates-jibneighbors.shp")

In [3]:
n_draw_samples = 10
unif_sample = lhs(2, n_draw_samples)

In [4]:
with tqdm_joblib(tqdm(desc="Processing perturbations", total=n_draw_samples, position=0, leave=True)) as progress_bar:
    all_perturb_glaciers = Parallel(n_jobs=10)(
        delayed(compute_perturbation)(data_url, ogr_url,
                                       perturbation=perturb, sample=unif_sample[perturb, :], 
                                       total_time=1_000, dt=1, reverse=True)
                for perturb in range(n_draw_samples))
    del progress_bar

all_perturb_glaciers = pd.concat(all_perturb_glaciers).reset_index(drop=True)

Processing perturbations:   0%|          | 0/10 [00:00<?, ?it/s]

Integration trajectory:  88%|████████▊ | 882/1000 [00:00<00:00, 1086.75it/s]
Integration trajectory: 1001it [00:00, 1139.09it/s]                         

Integration trajectory:  95%|█████████▍| 949/1000 [00:00<00:00, 1167.59it/s]
Integration trajectory: 1001it [00:00, 1139.45it/s]                         

Integration trajectory: 1001it [00:00, 1099.87it/s]                         
Integration trajectory: 1001it [00:00, 1163.03it/s]                         

Integration trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
Integration trajectory: 1001it [00:00, 1091.41it/s]                         

Integration trajectory: 1001it [00:00, 1085.91it/s]                         

Integration trajectory: 1001it [00:00, 1125.68it/s]                         
Integration trajectory: 1001it [00:00, 1088.05it/s]                         

Integration trajectory: 1001it [00:00, 1100.90it/s]                         
Integration trajectory: 1001it [00:00, 1096.68it/s]                         

Inte

Integration trajectory:  78%|███████▊  | 775/1000 [00:00<00:00, 1130.26it/s]

Integration trajectory:  83%|████████▎ | 832/1000 [00:00<00:00, 1112.16it/s]
Integration trajectory: 1001it [00:00, 1129.52it/s]                         
Integration trajectory: 1001it [00:00, 1115.25it/s]                         

Integration trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
Integration trajectory: 1001it [00:00, 1093.13it/s]                         

Integration trajectory:  92%|█████████▏| 920/1000 [00:00<00:00, 1024.40it/s]
Integration trajectory: 1001it [00:00, 1150.14it/s]                         
Integration trajectory: 1001it [00:00, 1140.69it/s]                         
Integration trajectory: 1001it [00:00, 1095.79it/s]                         

Integration trajectory: 100%|██████████| 1000/1000 [00:00<00:00, 1077.11it/s]
Integration trajectory: 1001it [00:00, 1080.85it/s]                          [A
Integration trajectory: 1001it [00:00, 1087.00it/s]                         
Int

Integration trajectory:  80%|███████▉  | 799/1000 [00:00<00:00, 1136.65it/s]
Integration trajectory: 1001it [00:00, 1121.77it/s]                         

Integration trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
Integration trajectory: 1001it [00:00, 1057.31it/s]                         
Integration trajectory: 1001it [00:00, 1103.13it/s]                         

Integration trajectory:  92%|█████████▏| 915/1000 [00:00<00:00, 1145.90it/s]
Integration trajectory: 1001it [00:00, 1125.58it/s]                         

Integration trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
Integration trajectory: 1001it [00:00, 1118.58it/s]                         

Integration trajectory: 1001it [00:00, 1089.68it/s]                         
Integration trajectory: 1001it [00:00, 1109.14it/s]                         
Integration trajectory: 1001it [00:00, 1111.11it/s]                         
Integration trajectory: 1001it [00:00, 1093.74it/s]                         

Integration trajec

Integration trajectory:  89%|████████▊ | 886/1000 [00:00<00:00, 1115.18it/s]
Integration trajectory: 1001it [00:00, 1081.47it/s]                         

Integration trajectory:  89%|████████▊ | 886/1000 [00:00<00:00, 1066.54it/s]
Integration trajectory: 1001it [00:00, 1112.94it/s]                         

Integration trajectory:  90%|████████▉ | 896/1000 [00:00<00:00, 1082.95it/s]
Integration trajectory: 1001it [00:00, 1082.95it/s]                         

Integration trajectory:  99%|█████████▉| 994/1000 [00:00<00:00, 1047.63it/s]
Integration trajectory: 1001it [00:00, 1076.95it/s]                         
Integration trajectory: 1001it [00:00, 1050.87it/s]                         
Integration trajectory: 1001it [00:00, 1116.75it/s]                         
Integration trajectory: 1001it [00:00, 1144.48it/s]                         
Integration trajectory: 1001it [00:00, 1122.13it/s]                         
Integration trajectory: 1001it [00:00, 1103.57it/s]                      

...

Integration trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
Integration trajectory: 1001it [00:00, 1063.06it/s]                         [A

Integration trajectory:  21%|██        | 209/1000 [00:00<00:00, 1051.22it/s]
Integration trajectory: 1001it [00:01, 960.27it/s]                         ]

Integration trajectory: 1001it [00:00, 1058.78it/s]                         [A

Integration trajectory:  21%|██▏       | 214/1000 [00:00<00:00, 1071.09it/s]
Integration trajectory: 1001it [00:01, 984.14it/s]                         

Integration trajectory: 1001it [00:01, 973.54it/s]                         ]

Integration trajectory:  99%|█████████▉| 991/1000 [00:01<00:00, 1005.81it/s]
Integration trajectory: 1001it [00:01, 981.61it/s]                          [A

Integration trajectory:  64%|██████▍   | 641/1000 [00:00<00:00, 982.95it/s]
Integration trajectory: 1001it [00:00, 1045.78it/s]                         

Integration trajectory:  82%|████████▏ | 822/1000 [00:00<00:00, 953.45it/s]
I

Integration trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
Integration trajectory: 1001it [00:00, 1018.92it/s]                         A

Processing Kangilerngata Sermia:  67%|██████▋   | 4/6 [00:13<00:05,  2.91s/it]
Integration trajectory: 1001it [00:01, 981.74it/s]                         ]

Integration trajectory:  43%|████▎     | 426/1000 [00:00<00:00, 997.66it/s]

Integration trajectory:  74%|███████▍  | 739/1000 [00:00<00:00, 1011.08it/s]
Integration trajectory: 1001it [00:00, 1007.51it/s]                         

Integration trajectory:   9%|▉         | 89/1000 [00:00<00:01, 875.41it/s]
Integration trajectory: 1001it [00:01, 993.38it/s]                         ]

Integration trajectory:  96%|█████████▌| 960/1000 [00:00<00:00, 1059.12it/s]
Integration trajectory: 1001it [00:00, 1049.15it/s]                         

Integration trajectory: 1001it [00:00, 1036.86it/s]                         

Integration trajectory:  74%|███████▍  | 743/1000 [00:00<00:00, 945.24it/s]
Integ

Integration trajectory:  75%|███████▌  | 750/1000 [00:00<00:00, 1054.14it/s]
Integration trajectory: 1001it [00:00, 1026.06it/s]                         

Integration trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
Integration trajectory: 1001it [00:00, 1045.85it/s]                         [A

Integration trajectory:  96%|█████████▌| 961/1000 [00:00<00:00, 1022.88it/s]
Integration trajectory: 1001it [00:00, 1029.06it/s]                         

Integration trajectory:  68%|██████▊   | 676/1000 [00:00<00:00, 941.60it/s]
Integration trajectory: 1001it [00:01, 997.01it/s]                            

Integration trajectory:  82%|████████▏ | 824/1000 [00:00<00:00, 1007.94it/s]
Integration trajectory: 1001it [00:00, 1016.45it/s]                        ]
Processing Sermeq Kujalleq:  83%|████████▎ | 5/6 [00:15<00:02,  2.63s/it]     
Integration trajectory: 1001it [00:01, 986.57it/s]                         

Integration trajectory:  98%|█████████▊| 978/1000 [00:00<00:00, 1076.65it/s]
In

Integration trajectory:  99%|█████████▉| 993/1000 [00:01<00:00, 987.64it/s]
Integration trajectory: 1001it [00:01, 975.67it/s]                         ]

Integration trajectory:  96%|█████████▌| 962/1000 [00:00<00:00, 1025.40it/s]
Integration trajectory: 1001it [00:00, 1009.89it/s]                        ]

Integration trajectory: 1001it [00:00, 1033.79it/s]                         

Integration trajectory:  84%|████████▍ | 842/1000 [00:00<00:00, 1036.16it/s]
Integration trajectory: 1001it [00:01, 997.96it/s]                         ]
Integration trajectory: 1001it [00:01, 972.57it/s]                         

Integration trajectory:  88%|████████▊ | 878/1000 [00:00<00:00, 1051.16it/s]
Integration trajectory: 1001it [00:00, 1033.86it/s]                         [A

Integration trajectory:  98%|█████████▊| 984/1000 [00:00<00:00, 1050.21it/s]
Integration trajectory: 1001it [00:01, 997.97it/s]                          

Integration trajectory: 1001it [00:00, 1041.75it/s]                   

Integration trajectory:  99%|█████████▉| 988/1000 [00:00<00:00, 1062.41it/s]
Integration trajectory: 1001it [00:00, 1046.56it/s]                         

Integration trajectory:  87%|████████▋ | 866/1000 [00:00<00:00, 997.08it/s]
                                                                         /s]
Integration trajectory:  63%|██████▎   | 631/1000 [00:00<00:00, 1043.00it/s]
Integration trajectory: 1001it [00:00, 1027.01it/s]                         
Integration trajectory: 1001it [00:00, 1033.90it/s]                         

Integration trajectory:  88%|████████▊ | 885/1000 [00:00<00:00, 984.32it/s]
                                                                         /s]
Integration trajectory:  80%|███████▉  | 795/1000 [00:00<00:00, 1067.54it/s]
                                                                         /s]
Integration trajectory: 1001it [00:00, 1025.96it/s]                         

Integration trajectory:  77%|███████▋  | 768/1000 [00:00<00:00, 1019.95it/s

In [5]:
from tqdm.auto import trange